In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import Sequence, plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

In our classes we will use TensorFlow and Keras, however, a project can be done with any technology you want. You can even use pure NumPy and implement neural networks by yourself or write it in C or Assembler.

TensorFlow by default allocates nearly all available memory on the GPU. It's wise to set the memory_growth parameter then it will take just as much memory as needed.

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, enable=True)

Before we jump to neural networks let's start with a single perceptron. It's a simple mathematical model that calculates a weighted sum and applies a selected function called an activation function.

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(2,))) 
# that's how we create a single layer. First parameter specifies how many neurons do we want, since we want to have
# a single perceptron we use 1. Then we specify the input shape so bascially the number of variables 
model.summary()

Above we created a simple neural network with just one neuron accepting input consisting of two values. As you can see such a network/perceptron has 3 parameters. Two of them are the weights associated with each input. Additionally, there is one more parameter called bias. It can be treated as a weight associated with an additional constant input equal to one.

The formula for a single perceptron looks as follows.
$$ \sum (x_i*w_i) + bias$$

Let's check if it really works

In [ ]:
model.weights

Above you should see two weights as kernel and one bias. These weights are randomly initialized. Let's see if the formula is correct.

In [ ]:
x = np.array([[2,1]]) # dummy input for calculations
model.predict(x), x @ model.weights[0].numpy() + model.weights[1].numpy()

Method predict simply run neural network with given input.

Seems it works since the results are the same, btw @ is an operator for matrix multiplication. If you don't trust it you can write it step by step with scalar multiplication.

OK but you mentioned something about an activation function.

That's right, here we just used linear function f(x) = x, but in general, you can use whatever function you want e.g. tanh. It's applied after the weighted sum. So let's update our formula and try it once again.
$$ f(\sum (x_i*w_i) + bias)$$

In [ ]:
model = Sequential()
model.add(Dense(1, 'tanh', input_shape=(2,))) # the second parameter specifies the activation function
model.summary()

In [ ]:
model.weights

In [ ]:
model.predict(x), x @ model.weights[0].numpy() + model.weights[1].numpy()

Now result of predict is different than our manual computations. That's because in predict a tanh activation function is used. Let's add tanh to our formula and see if the results match

In [ ]:
np.tanh(x @ model.weights[0].numpy() + model.weights[1].numpy())

The results might not be exactly the same due to numerical errors and the implementation of tanh, but it's clear it works.

So why do we want to use different activation functions and which of them to use?

We will return to that question later on. 


Now let's talk about those weights and how to obtain them. 
Well, the idea is simple. Firstly you need a training set with predictors ($x$) and target ($y$). In our example let's try to predict the price of a flat ($y$) based on its area ($x$). Of course, more than one feature can and should be used e.g. floor, year, district, ..., but for clarity, let's use a single feature. 

In [ ]:
np.random.seed(41)
x = np.random.rand(200,1)*20+50

In [ ]:
np.random.seed(27)
y = .5*x + np.random.rand(*x.shape)*3 + np.log(x-49)*2

In [ ]:
plt.plot(x,y, '.')
plt.xlabel("$m^2$")
plt.ylabel("price in bitcoins")
plt.show()

Ok, we created a dataset (a fake one, but perfect for this exercise). Now let's create a perceptron that will solve this task. Which activation function should be used?

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,))) #now we have only one input parameter
model.summary()

In [ ]:
model.predict(x)

In [ ]:
plt.plot(x,y, '.', label='data')
plt.plot(np.sort(x,0), model.predict(np.sort(x,0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

That's our prediction. Probably we are not even close. It's not strange, weights are initialized in a randomized way. What we need now is a measure of this error called a loss function. Once it's defined, the process of training weights is just an optimization task and the goal is to find such weights that minimize the loss function. In fact, any optimization algorithm can be used here including random search - try a lot of random weights and choose the best ones. But in most cases, stochastic gradient descent is used - in this algorithm using derivative iteratively the weights are updated

The first idea might be simply to calculate $prediction - y$. Do you know why it fails? The problem is we will keep a sign so positive and negative errors will cancel each other. We can improve this approach by taking an absolute of it. This metric is called a mean absolute error and again there is one problem. It's not differentiable and some algorithms for optimization require the loss function to be differentiable. That's why we will use the so-called mean squared error $(prediction - y)^2$. It works more or less like mean absolute error + it's differentiable + lower impact of small errors and higher impact of big errors.


In [ ]:
mse = lambda x,y: ((y - x)**2).mean() # our loss function

I said you can use even random search so let's give it a try

In [ ]:
bestWeights = model.get_weights()
pred = model.predict(x, verbose=0)
bestError = mse(y, pred)
bestError

for _ in range(200):
    weights = [np.random.randn(1,1), np.random.randn(1)]
    model.set_weights(weights)
    pred = model.predict(x, verbose=0)
    err = mse(y, pred)
    if err < bestError:
        bestError = err
        bestWeights = model.get_weights()
        print(err)


In [ ]:
model.set_weights(bestWeights)
plt.plot(x,y, '.', label='data')
plt.plot(np.sort(x,0), model.predict(np.sort(x,0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Ok, it looks better but there should be a better way than a random search. Of course, there is. As I said before any optimization technique can be used here but in most cases, a version of stochastic gradient descent is used. Here we take advantage of a differentiable loss function and iteratively step by step change the weights in a direction set by gradient descent. 

There are two parameters related to optimizing neural network weights using this algorithm number of epochs and batch size. The batch size determines the number of rows used for one update of weights. The general rule is the higher the better. The number of epochs determines how many times we will go through the whole dataset in the training process. The total number of weights updates is size of data / batch size * number of epochs

This function is already implemented so we just have to call the fit function. Before we have to call a compile method where we say which loss function should be used and if we want to collect some additional metrics

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.compile(loss='mse', metrics='mae')
model.fit(x,y, epochs=300, batch_size=16)

How to adjust the number of epochs? When you look at the history you should be able to determine when it doesn't make much sense to continue the optimization process. At some point, there is no longer any progress. We can use so-called callbacks - functions executed at various stages of the model training e.g. after each epoch. We can monitor the progress and stop learning when it's not progressing anymore. Such a callback is already implemented and is called EarlyStopping. It can be used together with the ReduceLROnPlateau callback. Before stopping the learning phase first it reduces the learning rate allowing the reduction of loss function even more. 

In [ ]:
early = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='loss', patience=6)

model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.compile(loss='mse', metrics='mae')
model.fit(x,y, epochs=1000, batch_size=16, callbacks=[early, reduce])

In [ ]:
plt.plot(x,y, '.', label='data')
plt.plot(np.sort(x,0), model.predict(np.sort(x,0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

The result is not perfect but with such a model it cannot be better. This model can return only linear output and there isn't a line that will fit much better to the data we have. What we have to do is to extend the model. Let's create a neural network with more than one neuron. Typically in classical neural networks neurons are organized in layers. Outputs of the previous layer is an input for the next layer. All inputs are connected to each neuron from a given layer thus we call such architecture fully connected or dense.


Here we can return to the activation functions.

First, let's list the most popular functions

In [ ]:
print("linear")
r = np.linspace(-7,7)
plt.plot(r,r)
plt.grid()
plt.show()
print("tanh")
plt.plot(r, np.tanh(r))
plt.grid()
plt.show()
print("sigmoid")
plt.plot(r, 1/(1 + np.exp(-r)))
plt.grid()
plt.show()
print("relu")
plt.plot(r, np.where(r<0,0,r))
plt.grid()
plt.show()
print("leakyrelu")
plt.plot(r, np.where(r<0,r*.1,r))
plt.grid()
plt.show()

Now how to decide which one to use. As you probably noticed some functions have a limited range of values e.g. sigmoid returns values from 0 to 1 hence it's a perfect option when the output should be treated as a probability. 

In dense layers it doesn't make much sense to use linear activation. Do you know why? What's a linear transformation of linear transformation? 

Good default choice is relu

The process of weight selection uses the derivative of an activation function. As you probably noticed for most of them derivative is close to zero for big positive and negative values. That's why we want to stay close to 0. To achieve that it's a good practice to introduce a data scaling process. It's a good idea to try different options, but classical standardization should be enough in most cases

In [ ]:
ss_x = StandardScaler()
ss_y = StandardScaler()

transformed_x = ss_x.fit_transform(x)
transformed_y = ss_y.fit_transform(y)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) # 64 neurons in the first layer
model.add(Dense(1)) # no need to specify input shape. Why?

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='loss', patience=6)

model.fit(transformed_x,transformed_y, epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:
plt.plot(transformed_x,transformed_y, '.', label='data')
plt.plot(np.sort(transformed_x, 0), model.predict(np.sort(transformed_x, 0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

You might wonder why do we use sort for plotting the prediction. Well it's just to draw a nice line. When the points are in a random order and we plot lines between them it might be quite chaotic.

In [ ]:
plt.plot(transformed_x,transformed_y, '.', label='data')
plt.plot(transformed_x, model.predict(transformed_x), label='prediction')
plt.legend()
plt.grid()
plt.show()

To avoid it we can just plot points, but a line looks better

In [ ]:
plt.plot(transformed_x,transformed_y, '.', label='data')
plt.plot(transformed_x, model.predict(transformed_x), '.', label='prediction')
plt.legend()
plt.grid()
plt.show()

# Task 1
What's the best architecture? Play here https://playground.tensorflow.org/

# Task 2
Try different architectures. Use different number of layers, neurons, activation functions

Selecting the best number of layers and neurons is an optimization task itself. There are no strict rules. In general, you should increase the number of neurons as long as it improves the results. Be aware of overfitting - model learning noise in the data. Always split dataset to train, validation and training.

Remember even the most complex network works still like a single perceptron - it's just an equation with weights determined using an optimization algorithm.

Lifehack: since selecting architecture is an optimization task we can use an optimization algorithm to solve it. The most obvious representation would be a vector of size equal to the maximum number of layers we want and each element representing the number of neurons in such a layer. There are a few problems with this approach - it doesn't make sense to have neurons in the next layer if we have 0 neurons in the previous which is allowed in this representation. Removing 0 from the considered range is an option but then we will fix the number of layers. Also in most cases, it's not recommended to increase the number of neurons in the next layer. To avoid these issues we can change the representation. The first element will still be the number of neurons in the first layer, but all remainings will be the number from 0 to 1 meaning the fraction of neurons from the previous layer used in this layer. For example $[64, 1, .5, .5, 0, 0]$ would mean architecture with 4 layers with 64, 64, 32, and 16 neurons.

To avoid overfitting we split our dataset to training, validation, and test sets. It's a good practice in most cases to shuffle the data  since the initial order might have some problems. For example data can be sorted by a target value. Then if we split it in half training and tests se

In [ ]:
np.random.seed(31)
idx = np.arange(len(x))
np.random.shuffle(idx)
idx

In [ ]:
train = idx[:int(.8*len(x))]
val = idx[int(.8*len(x)):int(.9*len(x))]
test = idx[int(.9*len(x)):]

In [ ]:
train_x, train_y = transformed_x[train], transformed_y[train]
val_x, val_y = transformed_x[val], transformed_y[val]
test_x, test_y = transformed_x[test], transformed_y[test]

What's the problem here? Well it's not a big issue, but data from validation and test sets should not be used to determine scaling parameters. So the scaler should be fit on the training data and then applied to validation and test datasets. Can you do it now?

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(patience=15, restore_best_weights=True) # we don't specify monitor, by default it's val_loss
reduce = ReduceLROnPlateau(patience=6)

model.fit(train_x,train_y, validation_data=(val_x, val_y), epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:

plt.plot(test_x,test_y, '.', label='data')
plt.plot(np.sort(test_x, 0), model.predict(np.sort(test_x, 0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

There are two popular functions to create a neural network model. For simple architectures Sequential can be used. For more complex Model is a suitable choice.


In the sequential model, we assume layers are executed sequentially one by one. In Model, we can create architectures with different connections, multiple inputs or outputs.

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dense(32, activation='relu', input_shape=(1,))) 
model.add(Dense(16, activation='relu', input_shape=(1,))) 
model.add(Dense(1, activation='linear'))

In [ ]:
plot_model(model)

In [ ]:
inputLayer = Input(shape=(1,))
dense1 = Dense(64, activation='relu')(inputLayer)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(16, activation='relu')(dense2)
dense4 = Dense(1, activation='relu')(dense3)
model = Model(inputs=inputLayer, outputs=dense4)

In [ ]:
plot_model(model)

In [ ]:
inputLayer = Input(shape=(1,))
dense1 = Dense(64, activation='relu')(inputLayer)
dense2 = Dense(32, activation='relu')(dense1)
concat = Concatenate()([dense1, dense2])
dense3 = Dense(16, activation='relu')(concat)
dense4 = Dense(1, activation='relu')(dense3)
model = Model(inputs=inputLayer, outputs=dense4)

In [ ]:
plot_model(model)

# Task 3
Create a model with two inputs, first processed by three Dense layers, second by two, then concatenated, processed by two Dense layers and split it to two outputs


Data generators are useful tools for neural network training. It allows to create data separately for each batch. Thanks to that there is no need to load all the data to the memory, which can be an issue for example when dealing with images. Also when we don't have data but a function for a generation then instead of generating a fixed number of rows in advance a data generator can be used. It's also useful for the data augmentation process.

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, x, y, batch_size, shuffle=True):
        self.x = x
        self.y = y
        self.indexes = np.arange(len(y))
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch. 
        # During training and prediction this function will be called in range(0, __len__())
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, idx):
        X = np.empty((self.batch_size, 1))
        y = np.empty((self.batch_size), )

        for i, ID in enumerate(idx):
            # Store sample
            X[i,] = self.x[ID]

            # Store class
            y[i] = self.y[ID]

        return X, y

In [ ]:
trainGenerator = DataGenerator(train_x, train_y, 16)
valGenerator = DataGenerator(val_x, val_y, 1, shuffle=False)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(patience=15, restore_best_weights=True) # we don't specify monitor, by default it's val_loss
reduce = ReduceLROnPlateau(patience=6)

model.fit(trainGenerator, validation_data=valGenerator, epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:
plt.plot(test_x,test_y, '.', label='data')
plt.plot(np.sort(test_x, 0), model.predict(np.sort(test_x, 0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Of course, this generator doesn't make much sense since the data are already in the memory and we don't do anything with them. It's just a template you can use later when it's really needed.

# Task 4
Predict the quality of wine using neural network. 
 - Treat target as continuous.
 - Treat target as discrete classes.

In [ ]:
data = load_wine()

In [ ]:
x = data['data']
y = data['target']

In [ ]:
x

In [ ]:
y

In [ ]:
#try also this model
#check why and how it works
model = Sequential()
model.add(Dense(16, 'relu', input_shape=(13,)))
model.add(Dense(3, 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'])

# Task 5
Think how to use neural network for a time series with known and unknown length.